In [1]:
#Simple downloader for pixiv
from concurrent.futures import ThreadPoolExecutor

import os
import json
import shutil
import time
import requests
import sys

from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from Includes import SeleniumUtils
from Includes import GeneralFuncs

In [2]:
baselink = "https://www.pinterest.com/"

Mainfolder = "Pinterest\\"

timeout = 2

isLogin = False

In [5]:
driver = SeleniumUtils.GetChromeDriver()

正在创建网页实例...


In [6]:
print("正在载入Pinterest...")
driver.get(baselink)
elementNonlogin_present = EC.presence_of_element_located((By.XPATH, '//div[@data-test-id="unauth-header"]/div//button'))
WebDriverWait(driver, timeout).until(elementNonlogin_present)
# click login
header_elements = driver.find_elements(by=By.XPATH, value='//div[@data-test-id="unauth-header"]/div//button')
for element in header_elements:
    if element.text == '登录':
        login_element = element
        login_element.click()
        break
#check loading
elementNonlogin_present = EC.presence_of_element_located((By.XPATH, '//input[@type="email"]'))
WebDriverWait(driver, timeout).until(elementNonlogin_present)
# send username and keywords
username_element = driver.find_elements(by=By.XPATH, value='//input[@type="email"]')[0]
password_element = driver.find_elements(by=By.XPATH, value='//input[@type="password"]')[0]
login_button = driver.find_elements(by=By.XPATH,
                                    value='//div[@data-test-id="registerFormSubmitButton"]/button')[0]

username_element.send_keys("Email")
password_element.send_keys("PW")
login_button.click()

正在载入Pinterest...


In [23]:
#loginlink = driver.find_element(By.PARTIAL_LINK_TEXT, '登录')
#loginlink = driver.find_elements(By.XPATH, "//button[text()='登录']")
#loginlink[0].send_keys(Keys.ENTER)

<selenium.webdriver.remote.webelement.WebElement (session="ecd9e8b22e90a87f7eb31d166bc7e1b1", element="f89c1702-c67a-4790-b5b3-105994cd8d6a")>

In [4]:
#check loading
elementNonlogin_present = EC.presence_of_element_located((By.XPATH, '//input[@name="searchBoxInput"]'))
WebDriverWait(driver, timeout).until(elementNonlogin_present)

TimeoutException: Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x00EE5FD3+2187219]
	Ordinal0 [0x00E7E6D1+1763025]
	Ordinal0 [0x00D93E78+802424]
	Ordinal0 [0x00DC1C10+990224]
	Ordinal0 [0x00DC1EAB+990891]
	Ordinal0 [0x00DEEC92+1174674]
	Ordinal0 [0x00DDCBD4+1100756]
	Ordinal0 [0x00DECFC2+1167298]
	Ordinal0 [0x00DDC9A6+1100198]
	Ordinal0 [0x00DB6F80+946048]
	Ordinal0 [0x00DB7E76+949878]
	GetHandleVerifier [0x011890C2+2721218]
	GetHandleVerifier [0x0117AAF0+2662384]
	GetHandleVerifier [0x00F7137A+526458]
	GetHandleVerifier [0x00F70416+522518]
	Ordinal0 [0x00E84EAB+1789611]
	Ordinal0 [0x00E897A8+1808296]
	Ordinal0 [0x00E89895+1808533]
	Ordinal0 [0x00E926C1+1844929]
	BaseThreadInitThunk [0x76B500C9+25]
	RtlGetAppContainerNamedObjectPath [0x778C7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x778C7B1E+238]


In [7]:
#Search
search_element = driver.find_elements(by=By.XPATH, value='//input[@name="searchBoxInput"]')[0]
prompts = len(search_element.get_attribute("value")) + 2# Additional BS 
search_element.send_keys(Keys.BACKSPACE * prompts)
search_element.send_keys("Maid")
search_element.send_keys(Keys.ENTER)

In [ ]:
#check loading
elementNonlogin_present = EC.presence_of_element_located((By.XPATH, '//div[@data-test-id="non-story-pin-image"]/div/img'))
WebDriverWait(driver, timeout).until(elementNonlogin_present)

In [10]:
page_elements = driver.find_elements(By.XPATH, '//div[@data-test-id="non-story-pin-image"]/div/img')
len(page_elements)

10

In [30]:
#向下滚动，载入下一页
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
driver.execute_script(f"window.scrollTo({2 * 500}, "f"{ 2 * 500});")

In [31]:
driver.execute_script("window.scrollTo(0, 0);")

In [12]:
img_element = driver.find_elements(By.XPATH, '//div[@data-test-id="non-story-pin-image"]/div/img')

In [18]:
img_element[-1].accessible_name

'This contains an image of: ハヤブサ🐤🐤 on Twitter'

In [27]:
#extention not right
img_url = img_element[0].get_attribute("src")
original_url = img_url.replace('236x', 'originals')
imglink = original_url
original_url

'https://i.pinimg.com/originals/eb/83/50/eb83506648433a7fc5b19269d376a451.jpg'

In [14]:
img_element[-1].accessible_name

'This contains an image of: Premium Vector | Anime manga girl dressed as a maid'

In [ ]:
img_element = driver.find_elements(By.XPATH, '//div[@data-test-id="non-story-pin-image"]/div/img')

In [35]:
#name = "This contains an image of: "
name = img_element[-1].accessible_name
imgname = name.replace("This contains an image of: ","").replace("\\", "").replace("/", "").replace("|", "")
imgname = "".join( x for x in imgname if (x.isalnum() or x in "._- "))
postfix = "." + imglink.split(".")[-1]
if(imgname == ""):
    imgname = imglink.split("/")[-1].split(".")[0]
imgname

'ハヤブサ on Twitter'

In [36]:
GeneralFuncs.downloadImageToDiskBase(imglink, "./output/Pinterest/"+imgname +"."+postfix)

文件保存成功


In [5]:
url = "https://i.pinimg.com/originals/2f/51/f1/2f51f10e1c106c9f91a84ef23d4c7900.jpg"
response = requests.get(url, stream=True, timeout=10)
total_size = int(response.headers.get('content-length'))
total_size

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'